# Subset scenes, DEMs

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import shapely
import geopandas as gpd
import rasterio as rio
from rasterio.windows import Window
import rasterio.mask 
import shutil
import glob
from osgeo import gdal
import xarray as xr
from PIL import Image

In [4]:
# load, crop, and save rasters

# list interferograms
dirs = os.listdir('/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/asc_crop')

veloc_fn = '/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/veloc_crop/asc_veloc.tif'

# loop through interferograms
for i, idir in enumerate(dirs):
    cwd = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/asc_crop/{idir}'
    os.chdir(cwd)
    
    # select the line of sight displacement
    allfiles = os.listdir(cwd)
    for fn in allfiles:
        if fn[-12:] == 'los_disp.tif':
            og_tif = fn
        elif fn[-7:] == 'dem.tif':
            dem_tif = fn
    og_tif_fn = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/asc_crop/{idir}/{og_tif}'
    dem_tif_fn =  f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/asc_crop/{idir}/{dem_tif}'
    xsize, ysize = 250, 250  # define the test image size
    print("working on: ", og_tif)
    
    subset_number = 50  # define the number of subsets
    subsets = 0
    
    
    # loop until subsetting is finished
    while subsets < subset_number:
        
        # open src image
        og_src = rio.open(og_tif_fn)
        og_rs = og_src.read(1) # also open as np array
        
        dem_src = rio.open(dem_tif_fn)
        dem_rs = dem_src.read(1) # also open as np array
        
        veloc_src = rio.open(veloc_fn)
        veloc_rs = veloc_src.read(1) # also open as np array
        
        # create bounds for new origin location
        xmin, xmax = 0, og_src.width - xsize
        ymin, ymax = 0, og_src.height - ysize
        xoff, yoff = np.random.randint(xmin, xmax), np.random.randint(ymin, ymax)
        
        # only save if there are no nodata values in subset 
        if not 0.0 in og_rs[yoff:(yoff+ysize), xoff:(xoff+xsize)]:
            window = Window(xoff, yoff, xsize, ysize)
            og_transform = og_src.window_transform(window)
            dem_transform = dem_src.window_transform(window)
            veloc_transform = veloc_src.window_transform(window)

            # write subset
            og_profile = og_src.profile
            dem_profile = dem_src.profile
            veloc_profile = veloc_src.profile
            
            og_profile.update({
                'height': xsize,
                'width': ysize,
                'transform': og_transform})
            dem_profile.update({
                'height': xsize,
                'width': ysize,
                'transform': dem_transform})
            veloc_profile.update({
                'height': xsize,
                'width': ysize,
                'transform': veloc_transform})
            subsets+=1 # update index
        
            og_out_fn = f'/Users/qbren/Desktop/subsets/int/asc{subsets}_{og_tif}'
            dem_out_fn = f'/Users/qbren/Desktop/subsets/dem/asc{subsets}_{og_tif}'
            veloc_out_fn = f'/Users/qbren/Desktop/subsets/veloc/asc{subsets}_{og_tif}'

            with rasterio.open(og_out_fn, 'w', **og_profile) as og_dst:
                # Read the data from the window and write it to the output raster
                og_dst.write(og_src.read(window=window))
            og_src.close()
            
            with rasterio.open(dem_out_fn, 'w', **dem_profile) as dem_dst:
                # Read the data from the window and write it to the output raster
                dem_dst.write(dem_src.read(window=window))
            dem_src.close()
            
            with rasterio.open(veloc_out_fn, 'w', **veloc_profile) as veloc_dst:
                # Read the data from the window and write it to the output raster
                veloc_dst.write(veloc_src.read(window=window))
            veloc_src.close()

working on:  S1BB_20170515T010120_20170527T010121_VVP012_INT40_G_ueF_15DE_los_disp.tif
working on:  S1BB_20170527T010121_20170608T010122_VVP012_INT40_G_ueF_7895_los_disp.tif
working on:  S1BB_20170702T010123_20170714T010124_VVP012_INT40_G_ueF_B323_los_disp.tif
working on:  S1BB_20170714T010124_20170726T010125_VVP012_INT40_G_ueF_3554_los_disp.tif
working on:  S1BB_20170726T010125_20170807T010125_VVP012_INT40_G_ueF_F3ED_los_disp.tif
working on:  S1BB_20170807T010125_20170819T010126_VVP012_INT40_G_ueF_B043_los_disp.tif
working on:  S1BB_20170912T010127_20170924T010127_VVP012_INT40_G_ueF_4B64_los_disp.tif
working on:  S1BB_20180522T010127_20180603T010128_VVP012_INT40_G_ueF_E2AC_los_disp.tif
working on:  S1BB_20180627T010129_20180709T010130_VVP012_INT40_G_ueF_8921_los_disp.tif
working on:  S1BB_20180709T010130_20180721T010131_VVP012_INT40_G_ueF_C67C_los_disp.tif
working on:  S1BB_20180721T010131_20180802T010132_VVP012_INT40_G_ueF_B665_los_disp.tif
working on:  S1BB_20180802T010132_20180814T

## Combine ints and velocity to make training images

In [5]:
# first, adding ints to velocities to create training images 
int_fns = os.listdir('/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/subsets/int')

def list_tifs(my_fns):
    my_list = []
    for i in my_fns:
        if i[-4:] == '.tif':
            my_list.append(i)
    return my_list

int_list = list_tifs(int_fns)

In [6]:
for i, fn in enumerate(int_list):
    int_fn = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/subsets/int/{fn}'
    veloc_fn = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/subsets/veloc/{fn}'
    
    int_src = rio.open(int_fn)
    int_rs = int_src.read(1) # also open as np array
        
    veloc_src = rio.open(veloc_fn)
    veloc_rs = veloc_src.read(1) # also open as np array
    
    train = int_rs+veloc_rs
    
    out_fn = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/data/subsets/train/{fn}'
    
    im = Image.fromarray(train)
    im.save(out_fn)